In [30]:
import os
import re
import fitz  # PyMuPDF for PDF text extraction
from langchain.chat_models import ChatOpenAI
from langchain_groq import ChatGroq
from pdf2image import convert_from_path
from pylatex import Document
from dotenv import load_dotenv
import subprocess

In [31]:
# Load API key from .env file
load_dotenv()
api_key_groq = os.getenv("GROQ_API_KEY")

In [32]:
# Define directories
DATA_DIR = r"C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\Data"
OUTPUT_DIR = r"C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\final_output"


In [33]:

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text



In [34]:
prev_pdf_path = os.path.join(DATA_DIR, "prev_year_copy.pdf")
curr_pdf_path = os.path.join(DATA_DIR, "curr_year_report.pdf")

prev_text_path = os.path.join(DATA_DIR, "prev_text.txt")
curr_text_path = os.path.join(DATA_DIR, "curr_text.txt")

prev_year_text = extract_text_from_pdf(prev_pdf_path)
curr_year_text = extract_text_from_pdf(curr_pdf_path)

# Save for reference
with open(prev_text_path, "w", encoding="utf-8") as f:
    f.write(prev_year_text)

with open(curr_text_path, "w", encoding="utf-8") as f:
    f.write(curr_year_text)


In [35]:
import re

def replace_numbers_with_blanks(text):
    # Regex pattern to match numbers, decimals, percentages, and currency values
    pattern = r"\b\d+[\.,]?\d*\b|\b\d{1,3}(?:,\d{3})*\b|\$\d+[\.,]?\d*|\d+%"
    
    # Replace matched numbers with "__"
    blank_text = re.sub(pattern, "__", text)
    
    return blank_text

# Apply the function to remove numbers from previous year report
blank_prev_year_text = replace_numbers_with_blanks(prev_year_text)

blank_prev_year_text_path = os.path.join(OUTPUT_DIR, "blank_prev_year_text.txt")
# Save blank text (for debugging, optional)
with open(blank_prev_year_text_path, "w", encoding="utf-8") as f:
    f.write(blank_prev_year_text)

print("Numbers replaced with blanks successfully!")


Numbers replaced with blanks successfully!


In [36]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

### STEP 3: SET UP LLM WITH GROQ ###
llm = ChatGroq(model_name="llama-3.1-8b-instant", temperature=0.2, api_key=api_key_groq)

prompt_template = PromptTemplate(
    input_variables=["prev_report", "curr_report"],
    template="""
    You are a professional document editor. Your task is to **ONLY** replace missing values (`____`) 
    in the previous year's report using the corresponding values from the current year's report, 
    while **keeping the document structure, wording, and formatting exactly the same**.

    **Rules:**
    - Do **not** modify any words, sentences, section titles, or formatting.
    - Only replace `____` placeholders with the correct values from the current report.
    - If a value does not exist in the current report, keep `____` unchanged.
    - Ensure that the response is **formatted exactly like the original document**.

    ---
    **Previous Year Report (with missing values):**
    {prev_report}

    ---
    **Current Year Report (reference for values):**
    {curr_report}

    ---
    **Updated Report (with values filled in):**
    """
)

### STEP 4: RUN LLM TO FILL BLANKS ###
llm_chain = LLMChain(llm=llm, prompt=prompt_template)
response = llm_chain.invoke({"prev_report": blank_prev_year_text, "curr_report": curr_year_text})


[INFO] HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


In [37]:
# Define the output file path
output_txt_path = os.path.join(OUTPUT_DIR, "replaced.txt")

# Save the response to a text file
with open(output_txt_path, "w", encoding="utf-8") as file:
    file.write(response['text'])

print(f"✅ File saved successfully at: {output_txt_path}")


✅ File saved successfully at: C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\final_output\replaced.txt


Create a same formatted doc

In [38]:
import os
import fitz  # PyMuPDF for PDF extraction
import re
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from docx import Document
from pdf2docx import Converter  # Import for PDF to DOCX conversion
from comtypes.client import CreateObject  # For DOCX to PDF conversion

# Load API key from .env file
load_dotenv()
api_key_groq = os.getenv("GROQ_API_KEY")

# Define directories
DATA_DIR = r"C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\Data"
OUTPUT_DIR = r"C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\final_output"

# Input file paths
prev_pdf_path = os.path.join(DATA_DIR, "prev_year_copy.pdf")
curr_pdf_path = os.path.join(DATA_DIR, "curr_year_report.pdf")

# Output file paths
prev_docx_path = os.path.join(OUTPUT_DIR, "prev_year_copy.docx")
blank_prev_docx_path = os.path.join(OUTPUT_DIR, "blank_prev_year.docx")
filled_docx_path = os.path.join(OUTPUT_DIR, "filled_prev_year.docx")
filled_pdf_path = os.path.join(OUTPUT_DIR, "filled_prev_year.pdf")

In [39]:

### 1️⃣ Convert Previous Year PDF to DOCX ###
def pdf_to_docx(input_pdf, output_docx):
    print(f"🔄 Converting {input_pdf} to {output_docx}...")
    cv = Converter(input_pdf)
    cv.convert(output_docx, start=0, end=None)  # Convert all pages
    cv.close()
    print("✅ PDF converted to DOCX with formatting!")

pdf_to_docx(prev_pdf_path, prev_docx_path)

[INFO] Start to convert C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\Data\prev_year_copy.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...


🔄 Converting C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\Data\prev_year_copy.pdf to C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\final_output\prev_year_copy.docx...


[INFO] [3/4] Parsing pages...
[INFO] (1/1) Page 1
[INFO] [4/4] Creating pages...
[INFO] (1/1) Page 1
[INFO] Terminated in 0.45s.


✅ PDF converted to DOCX with formatting!


In [40]:
from docx import Document
import re

def replace_numbers_in_docx(input_docx, output_docx):
    doc = Document(input_docx)
    
    # Regex pattern to match numbers, percentages, and currency values
    number_pattern = re.compile(r"\b\d+[\.,]?\d*\b|\b\d{1,3}(?:,\d{3})*\b|\$\d+[\.,]?\d*|\d+%")

    for para in doc.paragraphs:
        for run in para.runs:  # Iterate through each formatted run
            if number_pattern.search(run.text):  # Check if the run contains numbers
                run.text = number_pattern.sub("__", run.text)  # Replace numbers only in that run

    doc.save(output_docx)
    print(f"✅ Replaced numbers in {input_docx}, saved as {output_docx} with formatting preserved.")

replace_numbers_in_docx(prev_docx_path, blank_prev_docx_path)


✅ Replaced numbers in C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\final_output\prev_year_copy.docx, saved as C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\final_output\blank_prev_year.docx with formatting preserved.


Replace values

In [41]:
import re
from docx import Document

# File Paths
txt_file_path = output_txt_path
docx_file_path = blank_prev_docx_path
output_docx_path = os.path.join(OUTPUT_DIR,"final.docx")

import shutil



backup_docx_path = docx_file_path.replace(".docx", "_backup.docx")
shutil.copy(docx_file_path, backup_docx_path)


'C:\\Users\\K4562517\\OneDrive - Saint-Gobain\\Desktop\\Document content transfer\\final_output\\blank_prev_year_backup.docx'

In [42]:
# Step 1: Extract only numbers from the .txt file using regex
with open(txt_file_path, "r", encoding="utf-8") as file:
    txt_content = file.read()

replaced_values = re.findall(r"\d+\.\d+|\d+", txt_content)  # Extract decimals and whole numbers
print("Extracted Values:", replaced_values)



Extracted Values: ['2024', '2024', '9.8', '24.7', '3.8', '15.1', '1.6', '2', '33', '3', '35', '2025', '10', '25', '2027']


In [43]:
import pprint
# Step 2: Load the Word Document
doc = Document(docx_file_path)

# Initialize Index to Track Replacements
value_index = 0  

# Step 3: Iterate Over All Paragraphs and Replace Placeholders
for para in doc.paragraphs:
    for run in para.runs:
        while "__" in run.text and value_index < len(replaced_values):  
            # Replace only one placeholder at a time
            run.text = run.text.replace("__", replaced_values[value_index], 1)
            value_index += 1  # Move to next value
doc.save(output_docx_path)



print("✅ All placeholders replaced successfully while preserving formatting!")
print(f"🔄 A backup of the original file has been saved at: {backup_docx_path}")
print(f"📄 Final document saved at: {output_docx_path}")

✅ All placeholders replaced successfully while preserving formatting!
🔄 A backup of the original file has been saved at: C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\final_output\blank_prev_year_backup.docx
📄 Final document saved at: C:\Users\K4562517\OneDrive - Saint-Gobain\Desktop\Document content transfer\final_output\final.docx


In [44]:
from docx2pdf import convert

# File Paths
pdf_output_path = os.path.join(OUTPUT_DIR,"final.pdf")  # Output PDF file

# Convert DOCX to PDF
convert(output_docx_path, pdf_output_path)

print("✅ PDF saved successfully with original formatting!")


100%|██████████| 1/1 [00:05<00:00,  5.25s/it]

✅ PDF saved successfully with original formatting!
